# Simple example I

### Installation instruction

It is recommended that you clone the qthought repository to your local machine and then run

in the qthought folder.
If you did not pip install qthought, you can use the following quick-fix by uncommenting and adapting to your local file path

In [1]:
#import sys
#import os
# to run the example, set the following path to the folder path of qthought on your machine
#sys.path.append(os.path.abspath('/Users/nuri/qthought/qthought'))

### Start of the example

This is a detailed explanation of how to write a simple protocol for two parties thought experiment. Prior to reading this, it is recommended to take a look at the PDF description file *simple_examples*.

First, we import the ProjectQ operations needed for the protocol. Alice and Bob share a Bell state; to create it, we need a Hadamard gate and a CNOT gate.
We also import *Protocol* and *ProtocolStep* classes to be able to define steps of the protocol; *QuantumSystem* to operate quantum systems of different dimensionality; *Agent* class and all functions from the *copennhagen_theory* module.
We don't import *consistency* rules here, as in this example agents don't have to look at each other's viewpoints and glue them together. For an example where they do, please refer to the Frauchiger-Renner thought experiment protocol.

In [2]:
import warnings
warnings.filterwarnings('ignore')

import qthought.utils as ut
from qthought.protocol import Protocol, ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import Agent
from qthought.interpretations.copenhagen_theory import *
from projectq.ops import H, CNOT

The first action of the protocol (at time $t=0$) is the initilization of two qubits (*alice_qubit* and *bob_qubit*) in a Bell state. After defining the action, we define the step of the protocol by specifying: domain of action; written description of the action, which will be used for printouts during the run; time of the step; and which action variable being described.

In the second action ($t=1$) Alice writes down the result to her memory, and here the *observe* function comes in play: first variable corresponds to the memory (*Alice_memory*), and the second variable - to the system being measured (*alice_qubit*). In the *step2* variable, we again specify the step by its domain - which now includes an object *Alice* of the class *Agent* with 1 memory qubit and 1 inference qubit.

Third action ($t=2$) shows how Alice makes an inference about Bob's measurement outcome. 

At $t=3$, Bob makes a measurement of his qubit, and at $t=4$ he writes it down to his memory.

In [3]:
def action1(qsys:QuantumSystem):
    H | qsys['alice_qubit']
    CNOT | (qsys["alice_qubit"], qsys["bob_qubit"])
    
step1 = ProtocolStep(domain={'Qubit': ['alice_qubit','bob_qubit']},
                     descr = 'Prepare a Bell state of Alices and Bobs qubit ',
                     time  = 0,
                     action = action1)


def action2(qsys:QuantumSystem):
    observe(qsys['Alice_memory'],qsys['alice_qubit'])
    
step2 = ProtocolStep(domain={'Qubit': 'alice_qubit',
                             'Agent(1,1)': 'Alice'},
                     descr = 'Alice observes s',
                     time  = 1,
                     action = action2)

def action3(qsys:QuantumSystem):
    qsys['Alice'].make_inference()
    
step3 = ProtocolStep(domain={'Qubit': 'alice_qubit',
                             'Agent(1,1)': 'Alice'},
                     descr = 'Alice makes an inference about Bobs measurement',
                     time  = 2,
                     action = action3)

def action4(qsys:QuantumSystem):
    observe(qsys['Bob_memory'],qsys['bob_qubit'])
    
step4 = ProtocolStep(domain={'Qubit': 'bob_qubit',
                             'AgentMemory(1)': 'Bob'},
                     descr = 'Bob observes s',
                     time  = 3,
                     action = action4)

Now we glue together all steps of the protocol.

In [4]:
p = Protocol()
p += step1
p += step2
p += step3
p += step4

Now we can print out the full protocol and all systems required to run it.

In [5]:
p

Step 0: Prepare a Bell state of Alices and Bobs qubit (t:0)
Step 1: Alice observes s(t:1)
Step 2: Alice makes an inference about Bobs measurement(t:2)
Step 3: Bob observes s(t:3)

Requirements: 
------------------------------
Qubit             ['alice_qubit', 'bob_qubit']
Agent(1,1)        ['Alice']
AgentMemory(1)    ['Bob']

We define an inference table of Alice reasoning about Bob. Here we use *forward_inference*, as Alice is reasoning about a later step; if it is the other way around, one should use *backward_inference* instead.

In [6]:
Alice_inference_table = TA = forward_inference(protocol=p, 
                                               subsys_x='Alice_memory', t_x=1, 
                                               subsys_y='Bob_memory', t_y=3, 
                                               silent=True)

Finally, we initialize the quantum system of the setting according to the requirements of the protocol, and set the inference table in no prediction state.

We are ready to run! (and to look up Alice's inference table)

In [7]:
qsys = QuantumSystem(p.get_requirements())
qsys["Alice"].set_inference_table(Alice_inference_table, no_prediction_state=1)
qsys['Alice'].prep_inference()
p.run(qsys)

Require Qubit alice_qubit
Require Qubit bob_qubit
Require Agent(1,1) Alice
Require AgentMemory(1) Bob
0 Prepare a Bell state of Alices and Bobs qubit  t:0
State:
0.71|0100000> + 0.71|0100011>
1 Alice observes s t:1
State:
0.71|0100000> + 0.71|0100111>
2 Alice makes an inference about Bobs measurement t:2
State:
0.71|0100000> + 0.71|0101111>
3 Bob observes s t:3
State:
0.71|0100000> + 0.71|1101111>


In [8]:
Alice_inference_table

In:(Alice_memory:t1)  |  Out: (Bob_memory:t3)
----------------------------------------------------
           0          |        [0]
           1          |        [1]

This corresponds to a theoretical observation due to quantum theory.